<a href="https://colab.research.google.com/github/vvmnnnkv/private-ai/blob/master/Section%202%20-%20Federated%20Learning%20Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Federated Learning Project

This project explores remote learning and aggregation of model weights.

"Server" only sees aggregated model.

In [0]:
# install dependency
!pip install syft

In [2]:
import torch
import syft as sy
import math
import pandas as pd

hook = sy.TorchHook(torch)

W0703 19:52:06.109340 139941403436928 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0703 19:52:06.127920 139941403436928 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
# load MNIST included with Colab
def mnist_to_torch(df, train=True):
  y = pd.get_dummies(df[0])
  X = df.drop(0, axis=1)
  X, y = torch.tensor(X.values).type(torch.float), torch.tensor(y.values).type(torch.float)
  return X, y

# Train & test datasets
X_train, y_train = mnist_to_torch(pd.read_csv("sample_data/mnist_train_small.csv", header=None))
X_test, y_test = mnist_to_torch(pd.read_csv("sample_data/mnist_test.csv", header=None))

num_train = X_train.size(0)
num_features = X_train.size(1)

print("Train size %d, test size: %d" % (num_train, y_test.size(0)))

Train size 20000, test size: 10000


In [0]:
# number of workers
num_workers = 3
# Create workers
workers = []
for i in range(num_workers):
  worker = sy.VirtualWorker(hook, id="worker_%d" % i)
  workers.append(worker)

In [0]:
# Split data and send chunk to each worker
fed_dataset = []
chunk_size = num_train // num_workers
for i in range(num_workers):
  start = i * chunk_size
  if i + 1 < num_workers:
    end = (i + 1) * chunk_size
  else:
    end = num_train
  fed_dataset.append((
      X_train[start:end].send(workers[i]), 
      y_train[start:end].send(workers[i])
  ))


In [6]:
print(workers)
print(fed_dataset)

[<VirtualWorker id:worker_0 #objects:2>, <VirtualWorker id:worker_1 #objects:2>, <VirtualWorker id:worker_2 #objects:2>]
[((Wrapper)>[PointerTensor | me:66185758196 -> worker_0:94614901028], (Wrapper)>[PointerTensor | me:71706664369 -> worker_0:32131384440]), ((Wrapper)>[PointerTensor | me:12279854652 -> worker_1:67993701178], (Wrapper)>[PointerTensor | me:88871755116 -> worker_1:597763287]), ((Wrapper)>[PointerTensor | me:79878815535 -> worker_2:90803368877], (Wrapper)>[PointerTensor | me:24469173731 -> worker_2:60846637558])]


In [0]:
# Create aggregator worker
aggregator = sy.VirtualWorker(hook, id="aggregator")


In [0]:
# Calculate mean for all parameters of list of models and set to target_model
def avg_params(source_models, target_model):
  # get params and emptify
  avg_dict = target_model.state_dict()
  for param, _ in avg_dict.items():
    avg_dict[param].zero_()
  
  # sum up params
  for _, m in source_models.items():
    m_dict = m[0].state_dict()
    for param, _ in m_dict.items():
      avg_dict[param] += m_dict[param]
  
  # calc avg
  for param, _ in avg_dict.items():
    avg_dict[param] /= len(source_models)
  
  # set
  target_model.load_state_dict(avg_dict)

# Federated training procedure
def fed_train(model, criteria, fed_dataset, test_dataset, aggregator, opt, avg_epochs = 10, worker_epochs = 30, lr=0.001):
  for global_epoch in range(avg_epochs):
    # copy latest model to workers
    fed_models = {}
    for X, y in fed_dataset:
      fed_model = model.copy().send(X.location)
      optimizer = opt(params=fed_model.parameters(), lr=lr)
      fed_models[fed_model.location.id] = (fed_model, optimizer)
    
    # train in parallel on workers
    for local_epoch in range(worker_epochs):
      losses = []
      for X, y in fed_dataset:
        fed_model, optimizer = fed_models[X.location.id]
        pred = fed_model(X)
        loss = criteria(pred, y)
        loss.backward()
        optimizer.step()
        loss = loss.get()
        losses.append(loss)
      print('Avg loss (%d/%d): %f' % (global_epoch, local_epoch, sum(losses) / len(losses)))
      
    # aggregate worker models on aggregator
    # move models to aggregator
    for _, fm in fed_models.items():
      fm[0].move(aggregator)

    # prepare avg model placeholder on aggregator
    avg_model = model.copy().send(aggregator)
    with torch.no_grad():
      # make model with average params on aggregator
      avg_params(fed_models, avg_model)
      # retrieve weights and apply to local model
      avg_model = avg_model.get()
      model.load_state_dict(avg_model.state_dict())
      # calculate accuracy on test set
      X_test, y_test = test_dataset
      y_pred = torch.softmax(model(X_test), dim=1)
      valid = (torch.argmax(y_pred, dim=1) == torch.argmax(y_test, dim=1)).sum()
      print('Accuracy: %f' % (float(valid) / float(y_test.size(0))))

  return model

    

In [9]:
# Define a simple MLP model (softmax is included in loss)
model = torch.nn.Sequential(
  torch.nn.Linear(num_features, 50),
  torch.nn.ReLU(),
  torch.nn.Linear(50, 10)
)
loss = torch.nn.modules.loss.BCEWithLogitsLoss()

# Train!
fed_train(model, loss, fed_dataset, (X_test, y_test), aggregator, torch.optim.SGD, 10, 20)




Avg loss (0/0): 6.511034
Avg loss (0/1): 2.853069
Avg loss (0/2): 2.754401
Avg loss (0/3): 2.743122
Avg loss (0/4): 2.623974
Avg loss (0/5): 2.333915
Avg loss (0/6): 2.259338
Avg loss (0/7): 2.097608
Avg loss (0/8): 1.591617
Avg loss (0/9): 1.129129
Avg loss (0/10): 0.943001
Avg loss (0/11): 0.873390
Avg loss (0/12): 0.860417
Avg loss (0/13): 0.879201
Avg loss (0/14): 0.832374
Avg loss (0/15): 0.713959
Avg loss (0/16): 0.606388
Avg loss (0/17): 0.550632
Avg loss (0/18): 0.522244
Avg loss (0/19): 0.501486
Accuracy: 0.569400
Avg loss (1/0): 0.484057
Avg loss (1/1): 0.464428
Avg loss (1/2): 0.433690
Avg loss (1/3): 0.401670
Avg loss (1/4): 0.373823
Avg loss (1/5): 0.350277
Avg loss (1/6): 0.328791
Avg loss (1/7): 0.308742
Avg loss (1/8): 0.290810
Avg loss (1/9): 0.278112
Avg loss (1/10): 0.274149
Avg loss (1/11): 0.277833
Avg loss (1/12): 0.282963
Avg loss (1/13): 0.283684
Avg loss (1/14): 0.280111
Avg loss (1/15): 0.277553
Avg loss (1/16): 0.277600
Avg loss (1/17): 0.275951
Avg loss (1/1

Sequential(
  (0): Linear(in_features=784, out_features=50, bias=True)
  (1): ReLU()
  (2): Linear(in_features=50, out_features=10, bias=True)
)